In [1]:
import pandas as pd
import re
import numpy as np
import seaborn as sns
sns.set()
import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib as mpl
mpl.rcParams['figure.dpi']= 160
import matplotlib.pyplot as plt
import os 


days_list={16, 18, 21, 21.5, 23, 23.5, 24, 25, 32, 33, 35, 35.5, 37, 37.5, 38, 39, 46, 47, 50, 50.5, 52, 52.5, 53, 54}

In [2]:
counts=pd.read_csv('../../datasets/gibson/raw_tables/counts.tsv',sep='\t', index_col=0)
counts.index.name=None
counts.head()

,1-D0AM,1-D0PM,1-D1AM,1-D1PM,1-D29AM,1-D29PM,1-D2AM,1-D2PM,1-D3AM,1-D3PM,...,M2-D8-2A,M2-D8-2B,M2-D8-3A,M2-D8-3B,M2-D9-1A,M2-D9-1B,M2-D9-2A,M2-D9-2B,M2-D9-3A,M2-D9-3B
ASV_1,0,9,9,53,2729,3232,449,871,749,717,...,1717,2128,2121,2348,3313,2911,3834,3104,3626,3341
ASV_2,43,32,65,90,9482,7161,397,2187,3357,3793,...,14958,13749,14356,17205,13423,11314,19051,15783,18266,12650
ASV_3,50,107,946,2962,17,27,4990,6481,6429,7640,...,17145,16980,17800,19574,23115,20850,27270,21064,26375,22036
ASV_4,47,228,1865,7008,953,1816,7132,4299,3885,4484,...,64,68,75,82,104,65,123,100,107,97
ASV_5,18,45,487,1077,2350,1613,1949,2197,2477,2388,...,451,464,484,511,657,614,828,586,725,634


In [3]:
taxonomy=pd.read_csv('../../datasets/gibson/raw_tables/rdp_species.tsv',sep='\t', index_col=0)
taxonomy=taxonomy.fillna('unknown')
taxonomy.head()

,sequence,Kingdom,Phylum,Class,Order,Family,Genus,Species
ASV_1,TACGGAGGATCCGAGCGTTATCCGGATTTATTGGGTTTAAAGGGAG...,Bacteria,Bacteroidetes,Bacteroidia,Bacteroidales,Bacteroidaceae,Bacteroides,cellulosilyticus/timonensis
ASV_2,TACAGAGGTCTCAAGCGTTGTTCGGAATCACTGGGCGTAAAGCGTG...,Bacteria,Verrucomicrobia,Verrucomicrobiae,Verrucomicrobiales,Akkermansiaceae,Akkermansia,unknown
ASV_3,TACGGAGGATCCGAGCGTTATCCGGATTTATTGGGTTTAAAGGGAG...,Bacteria,Bacteroidetes,Bacteroidia,Bacteroidales,Bacteroidaceae,Bacteroides,ovatus
ASV_4,TACGGAGGATCCGAGCGTTATCCGGATTTATTGGGTTTAAAGGGAG...,Bacteria,Bacteroidetes,Bacteroidia,Bacteroidales,Bacteroidaceae,Bacteroides,ovatus
ASV_5,TACGGAGGATCCGAGCGTTATCCGGATTTATTGGGTTTAAAGGGAG...,Bacteria,Bacteroidetes,Bacteroidia,Bacteroidales,Bacteroidaceae,Bacteroides,unknown


In [4]:
taxonomy_temp=taxonomy.copy()
taxonomy_temp.insert(8, 'ASV', pd.Series(np.array(taxonomy.index), index=taxonomy.index))
taxonomy_temp

,sequence,Kingdom,Phylum,Class,Order,Family,Genus,Species,ASV
ASV_1,TACGGAGGATCCGAGCGTTATCCGGATTTATTGGGTTTAAAGGGAG...,Bacteria,Bacteroidetes,Bacteroidia,Bacteroidales,Bacteroidaceae,Bacteroides,cellulosilyticus/timonensis,ASV_1
ASV_2,TACAGAGGTCTCAAGCGTTGTTCGGAATCACTGGGCGTAAAGCGTG...,Bacteria,Verrucomicrobia,Verrucomicrobiae,Verrucomicrobiales,Akkermansiaceae,Akkermansia,unknown,ASV_2
ASV_3,TACGGAGGATCCGAGCGTTATCCGGATTTATTGGGTTTAAAGGGAG...,Bacteria,Bacteroidetes,Bacteroidia,Bacteroidales,Bacteroidaceae,Bacteroides,ovatus,ASV_3
ASV_4,TACGGAGGATCCGAGCGTTATCCGGATTTATTGGGTTTAAAGGGAG...,Bacteria,Bacteroidetes,Bacteroidia,Bacteroidales,Bacteroidaceae,Bacteroides,ovatus,ASV_4
ASV_5,TACGGAGGATCCGAGCGTTATCCGGATTTATTGGGTTTAAAGGGAG...,Bacteria,Bacteroidetes,Bacteroidia,Bacteroidales,Bacteroidaceae,Bacteroides,unknown,ASV_5
...,...,...,...,...,...,...,...,...,...
ASV_1084,TACGGAGGATCCGAGCGTTATCCGGAATCATTGGGCGTAAAGAGGG...,Bacteria,Firmicutes,Erysipelotrichia,Erysipelotrichales,Erysipelotrichaceae,Coprobacillus,unknown,ASV_1084
ASV_1085,TACGTAGGGGGCAAGCGTTATCCGGATTTACTGGGTGTAAAGGGAG...,Bacteria,unknown,unknown,unknown,unknown,unknown,unknown,ASV_1085
ASV_1086,TACTTATGTTTCCCTCTTTCTCCTGATTTCCTGGGTGTAAAGGGAG...,Bacteria,Firmicutes,Clostridia,Clostridiales,Lachnospiraceae,unknown,unknown,ASV_1086
ASV_1087,TACGTAGGGTGCGAGCGTTAATCGGAATTACTGGGCGTAAAGGGTG...,Bacteria,Firmicutes,Clostridia,Clostridiales,Ruminococcaceae,unknown,unknown,ASV_1087


In [5]:
taxonomy_temp = taxonomy_temp.drop('sequence', axis=1)
taxonomy_temp

,Kingdom,Phylum,Class,Order,Family,Genus,Species,ASV
ASV_1,Bacteria,Bacteroidetes,Bacteroidia,Bacteroidales,Bacteroidaceae,Bacteroides,cellulosilyticus/timonensis,ASV_1
ASV_2,Bacteria,Verrucomicrobia,Verrucomicrobiae,Verrucomicrobiales,Akkermansiaceae,Akkermansia,unknown,ASV_2
ASV_3,Bacteria,Bacteroidetes,Bacteroidia,Bacteroidales,Bacteroidaceae,Bacteroides,ovatus,ASV_3
ASV_4,Bacteria,Bacteroidetes,Bacteroidia,Bacteroidales,Bacteroidaceae,Bacteroides,ovatus,ASV_4
ASV_5,Bacteria,Bacteroidetes,Bacteroidia,Bacteroidales,Bacteroidaceae,Bacteroides,unknown,ASV_5
...,...,...,...,...,...,...,...,...
ASV_1084,Bacteria,Firmicutes,Erysipelotrichia,Erysipelotrichales,Erysipelotrichaceae,Coprobacillus,unknown,ASV_1084
ASV_1085,Bacteria,unknown,unknown,unknown,unknown,unknown,unknown,ASV_1085
ASV_1086,Bacteria,Firmicutes,Clostridia,Clostridiales,Lachnospiraceae,unknown,unknown,ASV_1086
ASV_1087,Bacteria,Firmicutes,Clostridia,Clostridiales,Ruminococcaceae,unknown,unknown,ASV_1087


https://stackoverflow.com/questions/52761376/how-to-map-key-to-multiple-values-to-dataframe-column

In [6]:
def window(x):
    if x<=21.5:
        y=0.0
    elif x<=28.5:
        y=1.0
    elif x<=35.5:
        y=1.5
    elif x<=42.5:
        y=2.0     
    elif x<=50.5:
        y=2.5        
    else:
        y=3.0
    return y

In [7]:
index=pd.MultiIndex.from_frame(taxonomy_temp)

In [8]:
meta=pd.read_csv('../../datasets/gibson/raw_tables/metadata.tsv',sep='\t', index_col=0)
meta.index.name = 'sample'
meta

,subject,time
sample,,
1-D0AM,1,0.0
1-D0PM,1,0.5
1-D1AM,1,1.0
1-D1PM,1,1.5
1-D29AM,1,29.0
...,...,...
M2-D9-1B,M2-D9,1.0
M2-D9-2A,M2-D9,2.0
M2-D9-2B,M2-D9,3.0


In [9]:
meta.reset_index(inplace=True)
meta

,sample,subject,time
0,1-D0AM,1,0.0
1,1-D0PM,1,0.5
2,1-D1AM,1,1.0
3,1-D1PM,1,1.5
4,1-D29AM,1,29.0
...,...,...,...
334,M2-D9-1B,M2-D9,1.0
335,M2-D9-2A,M2-D9,2.0
336,M2-D9-2B,M2-D9,3.0
337,M2-D9-3A,M2-D9,4.0


In [10]:
meta['window']=meta['time'].apply(window)
meta[21:40]

,sample,subject,time,window
21,2-D18,2,18.0,0.0
22,2-D1AM,2,1.0,0.0
23,2-D1PM,2,1.5,0.0
24,2-D21AM,2,21.0,0.0
25,2-D21PM,2,21.5,0.0
26,2-D22AM,2,22.0,1.0
27,2-D22PM,2,22.5,1.0
28,2-D23AM,2,23.0,1.0
29,2-D23PM,2,23.5,1.0
30,2-D24,2,24.0,1.0


In [11]:
columns=pd.MultiIndex.from_frame(meta)
c_names=['sample', 'subject', 'time', 'window']
print(columns)
print(c_names)

MultiIndex([(  '1-D0AM',     '1',  0.0, 0.0),
            (  '1-D0PM',     '1',  0.5, 0.0),
            (  '1-D1AM',     '1',  1.0, 0.0),
            (  '1-D1PM',     '1',  1.5, 0.0),
            ( '1-D29AM',     '1', 29.0, 1.5),
            ( '1-D29PM',     '1', 29.5, 1.5),
            (  '1-D2AM',     '1',  2.0, 0.0),
            (  '1-D2PM',     '1',  2.5, 0.0),
            (  '1-D3AM',     '1',  3.0, 0.0),
            (  '1-D3PM',     '1',  3.5, 0.0),
            ...
            ('M2-D8-2A', 'M2-D8',  2.0, 0.0),
            ('M2-D8-2B', 'M2-D8',  3.0, 0.0),
            ('M2-D8-3A', 'M2-D8',  4.0, 0.0),
            ('M2-D8-3B', 'M2-D8',  5.0, 0.0),
            ('M2-D9-1A', 'M2-D9',  0.0, 0.0),
            ('M2-D9-1B', 'M2-D9',  1.0, 0.0),
            ('M2-D9-2A', 'M2-D9',  2.0, 0.0),
            ('M2-D9-2B', 'M2-D9',  3.0, 0.0),
            ('M2-D9-3A', 'M2-D9',  4.0, 0.0),
            ('M2-D9-3B', 'M2-D9',  5.0, 0.0)],
           names=['sample', 'subject', 'time', 'window'], lengt

In [12]:
df=pd.DataFrame(data=counts.to_numpy(), index=index, columns=columns)
df_main=df.loc[:,df.columns.get_level_values(2).isin(days_list)].groupby(level=['subject', 'window'], axis=1, dropna=False).sum()
df_main.head()

subject                                                                                                                         2  \
window                                                                                                                        0.0   
Kingdom  Phylum          Class            Order              Family          Genus       Species                     ASV            
Bacteria Bacteroidetes   Bacteroidia      Bacteroidales      Bacteroidaceae  Bacteroides cellulosilyticus/timonensis ASV_1  18955   
         Verrucomicrobia Verrucomicrobiae Verrucomicrobiales Akkermansiaceae Akkermansia unknown                     ASV_2  38210   
         Bacteroidetes   Bacteroidia      Bacteroidales      Bacteroidaceae  Bacteroides ovatus                      ASV_3  99959   
                                                                                                                     ASV_4   1528   
                                                                                         unknown                     ASV_5   5758   

subject                                                                                                                            \
window                                                                                                                        1.0   
Kingdom  Phylum          Class            Order              Family          Genus       Species                     ASV            
Bacteria Bacteroidetes   Bacteroidia      Bacteroidales      Bacteroidaceae  Bacteroides cellulosilyticus/timonensis ASV_1   1343   
         Verrucomicrobia Verrucomicrobiae Verrucomicrobiales Akkermansiaceae Akkermansia unknown                     ASV_2  83219   
         Bacteroidetes   Bacteroidia      Bacteroidales      Bacteroidaceae  Bacteroides ovatus                      ASV_3    537   
                                                                                                                     ASV_4     81   
                                                                                         unknown                     ASV_5   1374   

subject                                                                                                                            \
window                                                                                                                        1.5   
Kingdom  Phylum          Class            Order              Family          Genus       Species                     ASV            
Bacteria Bacteroidetes   Bacteroidia      Bacteroidales      Bacteroidaceae  Bacteroides cellulosilyticus/timonensis ASV_1  16504   
         Verrucomicrobia Verrucomicrobiae Verrucomicrobiales Akkermansiaceae Akkermansia unknown                     ASV_2  28212   
         Bacteroidetes   Bacteroidia      Bacteroidales      Bacteroidaceae  Bacteroides ovatus                      ASV_3    700   
                                                                                                                     ASV_4  25264   
                                                                                         unknown                     ASV_5   2910   

subject                                                                                                                            \
window                                                                                                                        2.0   
Kingdom  Phylum          Class            Order              Family          Genus       Species                     ASV            
Bacteria Bacteroidetes   Bacteroidia      Bacteroidales      Bacteroidaceae  Bacteroides cellulosilyticus/timonensis ASV_1  60643   
         Verrucomicrobia Verrucomicrobiae Verrucomicrobiales Akkermansiaceae Akkermansia unknown                     ASV_2   3877   
         Bacteroidetes   Bacteroidia      Bacteroidales      Bacteroidaceae  Bacteroides ovatus                      ASV_3    645   
       

In [13]:
df=pd.DataFrame(data=counts.to_numpy(), index=index, columns=columns)

df=df.groupby(level=['Phylum','Class','Order','Family'], dropna=False).sum()
df.head()

df_main=df.loc[:,df.columns.get_level_values(2).isin(days_list)].groupby(level=['subject', 'window'], axis=1, dropna=False).sum()


df_main_rel=df_main/df_main.sum()
#ax1 = df_main_rel.loc[top10].transpose().plot.bar(stacked=True)
#ax1.legend(loc='center left', bbox_to_anchor=(1, 0.5))
#plt.show()

index_out=df_main.index.get_level_values(0).values + ' ' + df_main.index.get_level_values(1).values + ' ' + df_main.index.get_level_values(2).values + ' ' + df_main.index.get_level_values(3).values
counts_out=pd.DataFrame(data=df_main.to_numpy(), index=index_out)
counts_out

,0,1,2,3,4,5,6,7,8,9,...,14,15,16,17,18,19,20,21,22,23
Actinobacteria Actinobacteria Bifidobacteriales Bifidobacteriaceae,0,0,2,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Actinobacteria Actinobacteria Geodermatophilales Geodermatophilaceae,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Actinobacteria Actinobacteria Propionibacteriales Nocardioidaceae,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Actinobacteria Actinobacteria Streptosporangiales Nocardiopsaceae,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Actinobacteria Coriobacteriia Coriobacteriales Coriobacteriaceae,14,4,0,0,0,0,16,26,0,0,...,0,0,0,0,14,52,7,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Proteobacteria Oligoflexia Bacteriovoracales Bacteriovoracaceae,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Proteobacteria unknown unknown unknown,1712,2,705,70,1683,11,121,40,677,1569,...,32,3027,129,924,1297,32,348,15,18,7
Verrucomicrobia Verrucomicrobiae Verrucomicrobiales Akkermansiaceae,38775,83553,39000,4171,11977,329,38564,73864,25715,18874,...,29762,16330,24116,10530,45825,69028,14161,19537,15711,962
Verrucomicrobia Verrucomicrobiae Verrucomicrobiales unknown,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [14]:
meta_out=df_main.columns.to_frame(index=None)
print(meta_out)

   subject  window
0        2     0.0
1        2     1.0
2        2     1.5
3        2     2.0
4        2     2.5
5        2     3.0
6        3     0.0
7        3     1.0
8        3     1.5
9        3     2.0
10       3     2.5
11       3     3.0
12       4     0.0
13       4     1.0
14       4     1.5
15       4     2.0
16       4     2.5
17       4     3.0
18       5     0.0
19       5     1.0
20       5     1.5
21       5     2.0
22       5     2.5
23       5     3.0


In [15]:
os.makedirs("_tmp", exist_ok=True)
counts_out.to_csv('_tmp/healthy_of_counts_test.csv')
print(counts_out)
meta_out.to_csv('_tmp/healthy_of_meta_test.csv')

                                                       0      1      2     3   \
Actinobacteria Actinobacteria Bifidobacteriales...      0      0      2     0   
Actinobacteria Actinobacteria Geodermatophilale...      0      0      0     0   
Actinobacteria Actinobacteria Propionibacterial...      0      0      0     0   
Actinobacteria Actinobacteria Streptosporangial...      0      0      0     0   
Actinobacteria Coriobacteriia Coriobacteriales ...     14      4      0     0   
...                                                   ...    ...    ...   ...   
Proteobacteria Oligoflexia Bacteriovoracales Ba...      0      0      0     0   
Proteobacteria unknown unknown unknown               1712      2    705    70   
Verrucomicrobia Verrucomicrobiae Verrucomicrobi...  38775  83553  39000  4171   
Verrucomicrobia Verrucomicrobiae Verrucomicrobi...      0      0      0     0   
unknown unknown unknown unknown                        27      0      5    25   

                           

In [16]:
df=pd.DataFrame(data=counts.to_numpy(), index=index, columns=columns)

df=df.groupby(level=['Phylum'], dropna=False).sum()
df.head()

df_main=df.loc[:,df.columns.get_level_values(2).isin(days_list)].groupby(level=['subject', 'window'], axis=1, dropna=False).sum()


index_out=df_main.index.get_level_values(0).values
counts_out=pd.DataFrame(data=df_main.to_numpy(), index=index_out)
counts_out

,0,1,2,3,4,5,6,7,8,9,...,14,15,16,17,18,19,20,21,22,23
Actinobacteria,17,11,4,0,0,10,19,27,1,0,...,0,0,0,2,14,57,12,0,0,0
Bacteroidetes,229280,40385,119056,162747,179199,135975,208794,56311,135543,139153,...,114317,134623,174935,157808,199365,57243,173289,158558,190629,137597
Chloroflexi,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Cyanobacteria/Chloroplast,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Euryarchaeota,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Firmicutes,35487,30832,38637,28187,17445,17488,40624,33996,29922,17598,...,25670,19918,14404,15465,47486,45151,41587,22939,16538,17196
Fusobacteria,1,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,3,0,0
Lentisphaerae,1,0,0,0,0,0,0,0,0,0,...,0,0,2,0,5,0,1,0,1,0
Proteobacteria,31368,13143,14207,79567,20261,22818,26900,19777,19065,26649,...,12765,56446,16257,11687,16679,9509,11409,66752,15099,23885
Verrucomicrobia,38775,83553,39000,4171,11977,329,38564,73864,25715,18874,...,29762,16330,24116,10530,45825,69028,14161,19537,15711,962


In [17]:
meta_out=df_main.columns.to_frame(index=None)
meta_out

,subject,window
0,2,0.0
1,2,1.0
2,2,1.5
3,2,2.0
4,2,2.5
5,2,3.0
6,3,0.0
7,3,1.0
8,3,1.5
9,3,2.0


In [18]:
counts_out.to_csv('_tmp/healthy_phylum_counts_test.csv')
meta_out.to_csv('_tmp/healthy_phylum_meta_test.csv')